In [ ]:
# pitch by pitch raw data pull

from pybaseball import statcast

pitch_by_pitch = statcast(start_dt='2019-03-25',end_dt='2019-08-06',team='CHC')
pitch_by_pitch.head(10)

In [ ]:
# list of field names

import pandas  
for col in pitch_by_pitch.columns: 
    print(col) 

In [ ]:
# game by game list of games and result

from pybaseball import schedule_and_record

schedule = schedule_and_record(2019, 'CHC')
schedule['W/L'].replace('W-wo','W', inplace=True)
schedule['W/L'].replace('L-wo','L', inplace=True)
schedule

In [ ]:
# total counts of values in selected field

import numpy
import pandas

schedule['W/L'].value_counts()

In [ ]:
# creating averages through groupbys and conditionals

home_diff = schedule[schedule['Home_Away'] == 'Home'].groupby('W/L', as_index=False)['R','RA'].mean()
away_diff = schedule[schedule['Home_Away'] == '@'].groupby('W/L', as_index=False)['R','RA'].mean()
total_diff = schedule.groupby('W/L')['R','RA'].mean()

total_diff

In [ ]:
# calculate aggregates of fields using a groupby

grouped = schedule.groupby(['Opp','W/L','Home_Away']).agg({'R': ['count', 'mean'], 'RA': ['mean']})
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
grouped

In [ ]:
# create a new field as a calculation and then average that column in a groupby

grouped['diff'] = grouped['R_mean']-grouped['RA_mean']
run_diff = grouped.groupby(['Opp','W/L'])['diff'].mean()
run_diff

In [ ]:
# subset the top HR hitters and bottom HR hitters, calculate averages of those
# groups, and calculate the differences 

import pandas
from pybaseball import batting_stats

data = batting_stats(start_season=2019, end_season=2019, qual=300, ind=0)
data.head(10)

hr_leaders = data.sort_values(['HR'], ascending=False)
hr_top10 = hr_leaders[['Name','AB','HR','Hard%','ISO','BABIP','WAR','Pull%','Cent%','Oppo%','FB%','FBv','O-Swing%','Z-Swing%']].head(10)
hr_bot10 = hr_leaders[['Name','AB','HR','Hard%','ISO','BABIP','WAR','Pull%','Cent%','Oppo%','FB%','FBv','O-Swing%','Z-Swing%']].tail(10)

top_means = hr_top10.mean()
bot_means = hr_bot10.mean()
diff = top_means - bot_means
diff

In [ ]:
# subset based on multiple conditions and sort

from pybaseball import bwar_bat
import numpy

bwar_data = bwar_bat()
bwar_data = bwar_data[(bwar_data['year_ID'] == 2019) & (bwar_data['team_ID'] == 'CHC')]
bwar_data[['name_common','G','PA','salary','WAR']].sort_values('WAR',ascending = False)

In [ ]:
# subset based on multiple conditions and sort

from pybaseball import bwar_pitch
import numpy

pwar_data = bwar_pitch()
pwar_data = pwar_data[(pwar_data['year_ID'] == 2019) & (pwar_data['team_ID'] == 'CHC')]
pwar_data = pwar_data[['name_common','G','GS','RA','xRA','BIP_perc','salary','WAR']].sort_values('WAR',ascending = False)
pwar_data

In [ ]:
# subset of best ERA pitchers with min 100 IP

from pybaseball import pitching_stats

pitch = pitching_stats(start_season=2019).sort_values('ERA')
pitch = pitch[pitch['IP'] > 100]
pitch[['Name','Team','ERA','WAR','IP','K']]

In [ ]:
# find player ids using last played filter

from pybaseball import playerid_lookup

player = playerid_lookup('Bryant')
player = player[player['mlb_played_last'] == 2019]
player

In [ ]:
# results for a batter in a game

from pybaseball import statcast_batter
import numpy
import pandas

bryant = statcast_batter(player_id=592178)
bryant = bryant['events'].value_counts()
bryant

In [ ]:
# trying to join to the player id to get pitches thrown by pitcher

from pybaseball import statcast_single_game, playerid_reverse_lookup
import numpy
import pandas

cubs_beat_reds = statcast_single_game(game_pk = 565277)
cubs_beat_reds_pitchers = numpy.unique(cubs_beat_reds[['pitcher']])

cubs_beat_reds_pitchers
players = playerid_reverse_lookup(player_ids = cubs_beat_reds_pitchers)

new_cubs_beat_reds = cubs_beat_reds.join(players.set_index('key_mlbam'),on='pitcher')
new_cubs_beat_reds = new_cubs_beat_reds.fillna(0)

new_cubs_beat_reds.describe(include='all')

# cant figure this out...
# grouped = new_cubs_beat_reds(['pitch_type','name_last','name_first']).size().unstack(fill_value=0)
# grouped